In [ ]:
#%pip install gensim
# %pip install torch
# %pip install pandas


  Using cached gensim-4.3.3.tar.gz (23.3 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached numpy-1.26.4.tar.gz (15.8 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [21 lines of output]
      + c:\Users\danny\rnn_258\Scripts\python.exe C:\Users\danny\AppData\Local\Temp\pip-install-75d82o3f\numpy_5774dff1de5f48ecac9648176d92d521\vendored-meson\meson\meson.py setup C:\Users\danny\AppData\Local\Temp\pip-install-75d82o3f\numpy_5774dff1de5f48ecac9648176d92d521 C:\Users\danny\AppData\Local\Temp\pip-install-75d82o3f\numpy_5774dff1de5f48ecac9648176d92d521\.mesonpy-_hvpcy3m -Dbuildtype=release -Db_ndebug=if-release -Db_vscrt=md --native-file=C:\Users\danny\AppData\Local\Temp\pip-install-75d82o3f\numpy_5774dff1de5f48ecac9648176d92d521\.mesonpy-_hvpcy3m\meson-python-native-file.ini
      The Meson build system
      Version: 1.2.99
      Source dir: C:\Users\danny\AppData\Local\Temp\pip-install-75d82o3f\numpy_5774dff1de5f48ecac9648176d92d521
      Build dir: C:\Users\danny\AppData\Local\Temp\pip-install-75d82o3f\numpy_5774dff1de5f

In [1]:
import pickle
import pandas as pd
import itertools
from collections import Counter
import numpy as np
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from gensim.models import word2vec
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from torch.nn.utils.rnn import pad_sequence
from collections import defaultdict
import ast
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import os
import string
import tempfile
import re

In [2]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\danny\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\danny\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\danny\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
NVIDIA GeForce RTX 3070


In [ ]:
def build_vocab(sentences):
    word_counts = Counter(itertools.chain(*sentences))
    vocabulary_inv = [x[0] for x in word_counts.most_common()]
    vocabulary = {x: i for i, x in enumerate(vocabulary_inv)}
    return word_counts, vocabulary, vocabulary_inv

In [ ]:
# A function used to learn word embeddings through Word2vec module
def get_embeddings(inp_data, vocabulary_inv, size_features=100,
                   mode='skipgram',
                   min_word_count=2,
                   context=5):
    model_name = "embedding"
    model_name = os.path.join(model_name)
    num_workers = 15
    downsampling = 1e-3
    print('Training Word2Vec model...')
    sentences = [[vocabulary_inv[w] for w in s] for s in inp_data]
    if mode == 'skipgram':
        sg = 1
        print('Model: skip-gram')
    elif mode == 'cbow':
        sg = 0
        print('Model: CBOW')
    embedding_model = word2vec.Word2Vec(sentences, workers=num_workers,
                                        sg=sg,
                                        vector_size=size_features,
                                        min_count=min_word_count,
                                        window=context,
                                        sample=downsampling)
    print("Saving Word2Vec model {}".format(model_name))
    embedding_weights = np.zeros((len(vocabulary_inv), size_features))
    for i in range(len(vocabulary_inv)):
        word = vocabulary_inv[i]
        if word in embedding_model.wv:
            embedding_weights[i] = embedding_model.wv[word]
        else:
            embedding_weights[i] = np.random.uniform(-0.25, 0.25,
                                                     embedding_model.vector_size)
    return embedding_weights

In [ ]:
def preprocess_df(df):
    stop_words = set(stopwords.words('english'))
    stop_words.add('would')
    translator = str.maketrans(string.punctuation, ' ' * len(string.punctuation))
    preprocessed_sentences = []
    for i, row in df.iterrows():
        sent = row["text"]
        sent_nopuncts = sent.translate(translator)
        words_list = sent_nopuncts.strip().split()
        filtered_words = [word for word in words_list if word not in stop_words and len(word) != 1] # also skip space from above translation
        preprocessed_sentences.append(" ".join(filtered_words))
    df["text"] = preprocessed_sentences
    return df

In [ ]:
def save_predictions(model, dataloader, device, output_path):
    model.eval()
    preds = []
    with torch.no_grad():
        for text, state_idx, feats in dataloader:
            text, state_idx, feats = text.to(device), state_idx.to(device), feats.to(device)
            outputs = model(text, state_idx, feats)
            predicted = torch.argmax(outputs, dim=1)
            preds.extend(predicted.cpu().numpy())

    df_out = pd.DataFrame({'label': preds})
    df_out.to_csv(output_path, index=False)

In [ ]:
class RestaurantRNN(nn.Module):
    def __init__(self, vocab_size, state_vocab_size, embed_dim, state_embed_dim, hidden_dim, num_structured_feats, num_classes, embedding_weights=None):
        super(RestaurantRNN, self).__init__()
        self.word_embedding = nn.Embedding(vocab_size, embed_dim)
        if embedding_weights is not None:
            self.word_embedding.weight.data.copy_(torch.tensor(embedding_weights))
        
        self.state_embedding = nn.Embedding(state_vocab_size, state_embed_dim)
        self.rnn = nn.GRU(embed_dim, hidden_dim, num_layers=2, batch_first=True)

        total_input_dim = hidden_dim + state_embed_dim + num_structured_feats
        self.fc = nn.Linear(total_input_dim, num_classes)

    def forward(self, text_input, state_input, structured_feats):
        embedded_text = self.word_embedding(text_input) 
        _, hidden = self.rnn(embedded_text)
        hidden = hidden[-1]

        state_emb = self.state_embedding(state_input)

        combined = torch.cat((hidden, state_emb, structured_feats), dim=1)
        output = self.fc(combined)
        return output

class RestaurantDataset(Dataset):
    def __init__(self, text_seqs, state_idxs, structured_feats, labels=None, id=None):
        self.text_seqs = torch.tensor(text_seqs, dtype=torch.long)
        self.state_idxs = torch.tensor(state_idxs, dtype=torch.long)
        self.structured_feats = torch.tensor(structured_feats, dtype=torch.float32)
        self.id = id
        
        if labels is not None:
            self.labels = torch.tensor(labels, dtype=torch.long)
        else:
            self.labels = None

    def __len__(self):
        return len(self.labels) if self.labels is not None else len(self.text_seqs)

    def __getitem__(self, idx):
        if self.labels is not None:
            return (
                self.text_seqs[idx], 
                self.state_idxs[idx], 
                self.structured_feats[idx], 
                self.labels[idx]
            )
        else:
            # For test set, no label
            return (
                self.text_seqs[idx], 
                self.state_idxs[idx], 
                self.structured_feats[idx],
            )
    
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        """
        Args:
            patience (int): How many epochs to wait after last improvement.
            min_delta (float): Minimum change to qualify as an improvement.
        """
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = float('inf')
        self.early_stop = False
        self.best_model_state = None

    def __call__(self, val_loss, model):
        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
            self.best_model_state = model.state_dict()
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

In [29]:
def safe_state_transform(state_list, le, unk_idx):
    idxs = []
    for s in state_list:
        try:
            idxs.append(le.transform([s])[0])
        except ValueError:
            idxs.append(unk_idx)
    return idxs

In [34]:
data_path = "C:/Users/danny/OneDrive/Documents/UCSD/DSC 258R/kaggle_proj/"

train_df = pd.read_csv(data_path + "train.csv")
final_test_df = pd.read_csv(data_path + "test.csv")

train_df["text"] = train_df["review"]
final_test_df["text"] = final_test_df["review"]
train_df = preprocess_df(train_df)
final_test_df = preprocess_df(final_test_df)

In [11]:
class_counts = train_df['label'].value_counts()

print(class_counts)

print(train_df.columns.tolist())
train_df.head()

print("Train DataFrame info:")
train_df.info()

final_test_df.head()

# Look at categorical features (not including the target)
# categorical_columns = X.select_dtypes(include=['object']).columns
# print(f"\nCategorical columns in train data: {categorical_columns}")

# # Inspect unique values of categorical columns
# for col in categorical_columns:
#     print(f"\nUnique values in {col}:")
#     print(X[col].value_counts())

label
american (traditional)    2680
mexican                   2217
italian                   2032
chinese                   1696
american (new)            1399
japanese                  1063
mediterranean              728
canadian (new)             484
thai                       483
asian fusion               362
Name: count, dtype: int64
['id', 'attributes.HappyHour', 'attributes.Ambience', 'hours.Tuesday', 'postal_code', 'attributes.AgesAllowed', 'attributes.GoodForDancing', 'attributes.OutdoorSeating', 'hours.Saturday', 'attributes.Corkage', 'longitude', 'name', 'attributes.BusinessAcceptsCreditCards', 'attributes.RestaurantsTableService', 'attributes.RestaurantsReservations', 'hours.Friday', 'attributes.RestaurantsPriceRange2', 'attributes.WiFi', 'attributes.ByAppointmentOnly', 'attributes.Music', 'attributes.NoiseLevel', 'attributes.BYOB', 'state', 'attributes.Alcohol', 'attributes.HasTV', 'attributes', 'attributes.BYOBCorkage', 'hours.Wednesday', 'hours.Sunday', 'attributes.Rest

,id,attributes.HappyHour,attributes.Ambience,hours.Tuesday,postal_code,attributes.AgesAllowed,attributes.GoodForDancing,attributes.OutdoorSeating,hours.Saturday,attributes.Corkage,...,attributes.AcceptsInsurance,attributes.RestaurantsDelivery,attributes.DietaryRestrictions,attributes.BusinessAcceptsBitcoin,address,attributes.GoodForKids,attributes.GoodForMeal,hours,review,text
0,0,NaN,"b""{'touristy': False, 'hipster': False, 'roman...",b'11:30-0:0',b'M5A 1T1',NaN,NaN,b'True',b'17:0-2:0',NaN,...,NaN,b'False',NaN,NaN,b'366 Queen Street E',b'False',"b""{'dessert': False, 'latenight': False, 'lunc...","{'Tuesday': '11:30-0:0', 'Wednesday': '11:30-0...",Overall wonderful experience. \n\nThe owner Fa...,Overall wonderful experience nThe owner Farley...
1,1,NaN,"b""{'romantic': False, 'intimate': False, 'clas...",b'12:0-21:0',b'M6R 2N2',NaN,NaN,b'True',b'14:0-21:0',NaN,...,NaN,b'False',NaN,NaN,b'414 Roncesvalles Avenue',b'True',"b""{'dessert': False, 'latenight': False, 'lunc...","{'Monday': '12:0-21:0', 'Tuesday': '12:0-21:0'...","VIBE: Nieghbourhood hole. TV playing sports, p...",VIBE Nieghbourhood hole TV playing sports patr...
2,2,NaN,"b""{'romantic': False, 'intimate': False, 'tour...",NaN,b'53704',NaN,NaN,b'False',b'11:30-21:0',NaN,...,NaN,NaN,NaN,NaN,b'2817 E Washington Ave',b'True',NaN,"{'Monday': '11:30-21:0', 'Wednesday': '11:30-2...","Viet House is very good. The food was fresh, ...",Viet House good The food fresh well cooked fla...
3,3,NaN,"b""{'romantic': False, 'intimate': False, 'clas...",b'17:0-21:30',b'T2E 6Z3',NaN,NaN,b'False',b'17:0-22:0',NaN,...,NaN,b'False',NaN,NaN,"b'2323 32 Avenue NE, Suite 108'",b'True',"b""{'dessert': False, 'latenight': False, 'lunc...","{'Monday': '0:0-0:0', 'Tuesday': '17:0-21:30',...",This is a really good place. Not truly authent...,This really good place Not truly authentic Tha...
4,4,b'False',"b""{'romantic': False, 'intimate': False, 'tour...",b'11:0-21:0',b'85085',NaN,b'False',b'True',b'11:0-22:0',NaN,...,NaN,b'False',NaN,NaN,b'2470 W Happy Valley Rd',b'True',"b""{'dessert': False, 'latenight': False, 'lunc...","{'Monday': '0:0-0:0', 'Tuesday': '11:0-21:0', ...",This place is great. Think fast food Italian s...,This place great Think fast food Italian style...


In [ ]:
X = train_df.drop(columns=["label"])
y = train_df['label']
X = X.dropna(axis=1, thresh=13144)
X = X.drop(columns=["business_id", "address", "id", "city", "postal_code", "name", "review"])

all_states = sorted(set(train_df['state'].unique()).union(set(final_test_df['state'].unique())))
state_to_idx = {state: i for i, state in enumerate(all_states)}
unknown_state_idx = len(state_to_idx)

def map_state_column(series, mapping, unk_idx):
    return series.apply(lambda x: mapping.get(x.decode() if isinstance(x, bytes) else x, unk_idx))

le = LabelEncoder()
X['state_idx'] = map_state_column(X['state'], state_to_idx, unknown_state_idx)
final_test_df['state_idx'] = map_state_column(final_test_df['state'], state_to_idx, unknown_state_idx)

X = X.drop(columns=["state"])

label_le = LabelEncoder()
y_encoded = label_le.fit_transform(y)
label_mapping = dict(zip(label_le.transform(label_le.classes_), label_le.classes_))

X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size = 0.1, random_state=42)

final_test_df = final_test_df[X.columns.intersection(final_test_df.columns).tolist() + ['id']]
final_test_df.head()

,longitude,review_count,is_open,stars,latitude,text,state_idx,id
0,-79.363792,7,0,4.5,43.655808,Overall wonderful experience nThe owner Farley...,8,0
1,-79.451277,24,1,3.0,43.651701,VIBE Nieghbourhood hole TV playing sports patr...,8,1
2,-89.344794,18,0,3.5,43.102887,Viet House good The food fresh well cooked fla...,13,2
3,-114.005667,52,1,3.0,51.080983,This really good place Not truly authentic Tha...,0,3
4,-112.113049,213,1,3.5,33.714826,This place great Think fast food Italian style...,1,4


In [36]:
print(X['state_idx'].max())
print(final_test_df['state_idx'].max())

13
13


In [37]:
# # tokenization
# tagged_data = [word_tokenize(_d) for i, _d in enumerate(train_df["text"])]
# # build vocabulary from tokenized data
# word_counts, vocabulary, vocabulary_inv = build_vocab(tagged_data)
# # use the above mapping to create input data
# inp_data = [[vocabulary[word] for word in text] for text in tagged_data]
# # get embedding vector
# embedding_weights = get_embeddings(inp_data, vocabulary_inv)


# tagged_train_data = [word_tokenize(_d) for i, _d in enumerate(train_df["text"])]
# tagged_test_data = [word_tokenize(_d) for i, _d in enumerate(test_df["text"])]

# embedding_dim = embedding_weights.shape[1]
# vocab_size = embedding_weights.shape[0]

# print(embedding_weights)
# print(tagged_train_data)

In [ ]:
tagged_train_data = [word_tokenize(text) for text in X_train["text"]]
tagged_val_data = [word_tokenize(text) for text in X_val["text"]]
tagged_test_data = [word_tokenize(text) for text in final_test_df["text"]]

word_counts, vocabulary, vocabulary_inv = build_vocab(tagged_train_data)

inp_data = [[vocabulary[word] for word in text] for text in tagged_train_data]

embedding_weights = get_embeddings(inp_data, vocabulary_inv, size_features=100, mode='skipgram')
vocab_size, embed_dim = embedding_weights.shape

train_seq = [[vocabulary[word] for word in text if word in vocabulary] for text in tagged_train_data]
val_seq = [[vocabulary[word] for word in text if word in vocabulary] for text in tagged_val_data]
test_seq = [[vocabulary[word] for word in text if word in vocabulary] for text in tagged_test_data]

train_text_seqs = pad_sequence([torch.tensor(x) for x in train_seq], batch_first=True, padding_value=0)
val_text_seqs = pad_sequence([torch.tensor(x) for x in val_seq], batch_first=True, padding_value=0)
test_text_seqs = pad_sequence([torch.tensor(x) for x in test_seq], batch_first=True, padding_value=0)

structured_cols = ["longitude", "latitude", "review_count", "is_open", "stars"]
train_feats = torch.tensor(X_train[structured_cols].values, dtype=torch.float32)
val_feats = torch.tensor(X_val[structured_cols].values, dtype=torch.float32)
structured_feats_test = torch.tensor(final_test_df[structured_cols].values, dtype=torch.float32)

train_state_idxs = torch.tensor(X_train["state_idx"].values, dtype=torch.long)
val_state_idxs = torch.tensor(X_val["state_idx"].values, dtype=torch.long)
test_state_idxs = torch.tensor(final_test_df["state_idx"].values, dtype=torch.long)

train_labels = torch.tensor(y_train, dtype=torch.long)
val_labels = torch.tensor(y_val, dtype=torch.long)

dummy_labels = torch.zeros(len(final_test_df), dtype=torch.long)

train_dataset = RestaurantDataset(train_text_seqs, train_state_idxs, train_feats, train_labels)
val_dataset = RestaurantDataset(val_text_seqs, val_state_idxs, val_feats, val_labels)
test_dataset = RestaurantDataset(
    text_seqs=test_text_seqs,
    state_idxs=test_state_idxs,
    structured_feats=structured_feats_test,
    labels=dummy_labels,
    id=torch.tensor(final_test_df["id"].values)  # dummy labels, if needed
)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

Training Word2Vec model...
Model: skip-gram
Saving Word2Vec model embedding


C:\Users\danny\AppData\Local\Temp\ipykernel_20360\3255774520.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.text_seqs = torch.tensor(text_seqs, dtype=torch.long)
C:\Users\danny\AppData\Local\Temp\ipykernel_20360\3255774520.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.state_idxs = torch.tensor(state_idxs, dtype=torch.long)
C:\Users\danny\AppData\Local\Temp\ipykernel_20360\3255774520.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.structured_feats = torch.tensor(structured_feats, dtype=torch.float32)
C:\Use

In [40]:
print("Max state_idx in train_loader:", max([s.max().item() for _, s, _, _ in train_loader]))
print("state_vocab_size:", len(state_to_idx) + 1)

Max state_idx in train_loader: 13
state_vocab_size: 15


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Model setup
early_stopping = EarlyStopping(patience=10, min_delta=0.01)
num_epochs = 10000

model = RestaurantRNN(
    vocab_size=vocab_size,
    state_vocab_size=len(state_to_idx) + 1,
    embed_dim=embed_dim,
    state_embed_dim=8,
    hidden_dim=128,
    num_structured_feats=train_feats.shape[1],
    num_classes=len(set(train_labels))
)
model.word_embedding.weight.data.copy_(torch.tensor(embedding_weights))
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
#     optimizer, mode='min', factor=0.5, patience=5, verbose=True
# )
criterion = nn.CrossEntropyLoss()

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for text, state_idx, feats, label in train_loader:
        text, state_idx, feats, label = text.to(device), state_idx.to(device), feats.to(device), label.to(device)

        output = model(text, state_idx, feats)
        loss = criterion(output, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)

    # Validation
    model.eval()
    val_loss = 0
    correct, total = 0, 0
    with torch.no_grad():
        for text, state_idx, feats, label in val_loader:
            text, state_idx, feats, label = text.to(device), state_idx.to(device), feats.to(device), label.to(device)
            preds = model(text, state_idx, feats)
            loss = criterion(preds, label)
            val_loss += loss.item()

            pred_classes = preds.argmax(dim=1)
            correct += (pred_classes == label).sum().item()
            total += label.size(0)

    avg_val_loss = val_loss / len(val_loader)
    val_acc = correct / total

    print(f"Epoch {epoch+1} - Loss: {avg_loss:.4f} - Val Acc: {correct / total:.4f}")

    early_stopping(avg_val_loss, model)

    if early_stopping.early_stop:
        print("Early stopping triggered")
        break

# After training ends, load the best model state:
model.load_state_dict(early_stopping.best_model_state)


Epoch 1 - Loss: 4.6568 - Val Acc: 0.2084
Epoch 2 - Loss: 2.0365 - Val Acc: 0.2327
Epoch 3 - Loss: 2.0130 - Val Acc: 0.2304
Epoch 4 - Loss: 1.9967 - Val Acc: 0.2357
Epoch 5 - Loss: 1.9857 - Val Acc: 0.2471
Epoch 6 - Loss: 1.9854 - Val Acc: 0.2441
Epoch 7 - Loss: 1.9770 - Val Acc: 0.2335
Epoch 8 - Loss: 1.9835 - Val Acc: 0.2418
Epoch 9 - Loss: 1.9763 - Val Acc: 0.2289
Epoch 10 - Loss: 1.9775 - Val Acc: 0.2373
Epoch 11 - Loss: 1.9765 - Val Acc: 0.2373
Epoch 12 - Loss: 1.9772 - Val Acc: 0.2418
Epoch 13 - Loss: 1.9702 - Val Acc: 0.2578
Epoch 14 - Loss: 1.9752 - Val Acc: 0.2289
Epoch 15 - Loss: 1.9682 - Val Acc: 0.2426
Epoch 16 - Loss: 1.9696 - Val Acc: 0.2433
Epoch 17 - Loss: 1.9688 - Val Acc: 0.2464
Epoch 18 - Loss: 1.9731 - Val Acc: 0.2418
Epoch 19 - Loss: 1.9642 - Val Acc: 0.2433
Epoch 20 - Loss: 1.9686 - Val Acc: 0.2251
Epoch 21 - Loss: 1.9719 - Val Acc: 0.2418
Epoch 22 - Loss: 1.9609 - Val Acc: 0.2403
Epoch 23 - Loss: 1.9608 - Val Acc: 0.2403
Epoch 24 - Loss: 1.9633 - Val Acc: 0.2335
E

<All keys matched successfully>

In [ ]:
from torch.utils.data import DataLoader, ConcatDataset
from torch.nn.utils.rnn import pad_sequence

full_df = pd.concat([X_train, X_val], ignore_index=True)
full_labels = np.concatenate([y_train, y_val])

tagged_full_data = [word_tokenize(text) for text in full_df["text"]]

word_counts_full, vocabulary_full, vocabulary_inv_full = build_vocab(tagged_full_data)

full_seq = [[vocabulary_full[word] for word in text if word in vocabulary_full] for text in tagged_full_data]

# Pad sequences
full_text_seqs = pad_sequence([torch.tensor(x) for x in full_seq], batch_first=True, padding_value=0)

structured_cols = ["longitude", "latitude", "review_count", "is_open", "stars"]
full_feats = torch.tensor(full_df[structured_cols].values, dtype=torch.float32)

full_state_idxs = torch.tensor(full_df["state_idx"].values, dtype=torch.long)

# 7. Prepare labels tensor for full dataset
full_labels_tensor = torch.tensor(full_labels, dtype=torch.long)

# --- New: Get embedding weights for full vocab ---
embedding_weights_full = get_embeddings(full_seq, vocabulary_inv_full, size_features=100, mode='skipgram')
#embedding_weights_full = torch.tensor(embedding_weights_full, dtype=torch.float32)
vocab_size, embed_dim = embedding_weights_full.shape  # Update vocab size and embedding dim accordingly

# 8. Create full dataset and dataloader
full_dataset = RestaurantDataset(full_text_seqs, full_state_idxs, full_feats, full_labels_tensor)

def collate_fn(batch):
    texts, states, feats, labels = zip(*batch)
    texts_padded = pad_sequence(texts, batch_first=True, padding_value=0)
    states = torch.stack(states)
    feats = torch.stack(feats)
    labels = torch.stack(labels)
    return texts_padded, states, feats, labels

full_train_loader = DataLoader(full_dataset, batch_size=64, shuffle=True, collate_fn=collate_fn)

Training Word2Vec model...
Model: skip-gram
Saving Word2Vec model embedding


C:\Users\danny\AppData\Local\Temp\ipykernel_20360\3255774520.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.text_seqs = torch.tensor(text_seqs, dtype=torch.long)
C:\Users\danny\AppData\Local\Temp\ipykernel_20360\3255774520.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.state_idxs = torch.tensor(state_idxs, dtype=torch.long)
C:\Users\danny\AppData\Local\Temp\ipykernel_20360\3255774520.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.structured_feats = torch.tensor(structured_feats, dtype=torch.float32)
C:\Use

In [42]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_classes = len(set(full_labels))  # should be same as model num_classes
vocab_size_full, embed_dim_full = embedding_weights_full.shape
early_stopping = EarlyStopping(patience=10, min_delta=0.01)
num_epochs_full = 10000
#num_states = len(le.classes_) + 1

model_full = RestaurantRNN(
    vocab_size=vocab_size_full,
    state_vocab_size=len(state_to_idx) + 1,
    embed_dim=embed_dim_full,
    state_embed_dim=8,
    hidden_dim=128,
    num_structured_feats=train_feats.shape[1],
    num_classes=num_classes
)
model_full.word_embedding.weight.data.copy_(torch.tensor(embedding_weights_full))
model_full.to(device)

optimizer = torch.optim.Adam(model_full.parameters(), lr=1e-3)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
#     optimizer, mode='min', factor=0.5, patience=5, verbose=True
# )
criterion = nn.CrossEntropyLoss()

for epoch in range(num_epochs_full):
    model_full.train()
    total_loss = 0
    for text, state_idx, feats, label in full_train_loader:
        text, state_idx, feats, label = text.to(device), state_idx.to(device), feats.to(device), label.to(device)
        
        optimizer.zero_grad()
        output = model_full(text, state_idx, feats)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    avg_loss = total_loss / len(full_train_loader)
    print(f"Full training epoch {epoch+1}, Loss: {avg_loss:.4f}")

    # Call early stopping
    early_stopping(avg_loss, model_full)

    if early_stopping.early_stop:
        print("Early stopping triggered")
        break

Full training epoch 1, Loss: 2.9138
Full training epoch 2, Loss: 2.0647
Full training epoch 3, Loss: 1.9642
Full training epoch 4, Loss: 1.3344
Full training epoch 5, Loss: 0.9103
Full training epoch 6, Loss: 0.6856
Full training epoch 7, Loss: 0.5251
Full training epoch 8, Loss: 0.4019
Full training epoch 9, Loss: 0.2923
Full training epoch 10, Loss: 0.1943
Full training epoch 11, Loss: 0.1269
Full training epoch 12, Loss: 0.0875
Full training epoch 13, Loss: 0.0626
Full training epoch 14, Loss: 0.0525
Full training epoch 15, Loss: 0.0449
Full training epoch 16, Loss: 0.0357
Full training epoch 17, Loss: 0.0250
Full training epoch 18, Loss: 0.0223
Full training epoch 19, Loss: 0.0233
Full training epoch 20, Loss: 0.0287
Full training epoch 21, Loss: 0.0203
Full training epoch 22, Loss: 0.0173
Full training epoch 23, Loss: 0.0174
Full training epoch 24, Loss: 0.0111
Full training epoch 25, Loss: 0.0203
Full training epoch 26, Loss: 0.0136
Full training epoch 27, Loss: 0.0071
Full train

In [46]:
# Prediction function
def save_predictions(model, dataloader, device, output_path, label_mapping, id_series):

    model.eval()
    preds = []
    with torch.no_grad():
        for text, state_idx, feats, _ in dataloader:  # Ignore dummy label
            text, state_idx, feats = text.to(device), state_idx.to(device), feats.to(device)
            outputs = model(text, state_idx, feats)
            predicted = torch.argmax(outputs, dim=1)
            preds.extend(predicted.cpu().numpy())

    pred_labels = [label_mapping[p] for p in preds]
    df_out = pd.DataFrame({'Id': id_series.values, 'label': pred_labels})
    df_out.to_csv(output_path, index=False)

# Save predictions to file
output_path = "/Users/danny/OneDrive/Documents/UCSD/DSC 258R/kaggle_proj/rnn_preds/predicted.csv"
save_predictions(model_full, test_loader, device, output_path, label_mapping, final_test_df["id"])

In [ ]:
# def save_predictions(model, dataloader, device, output_path):
#     model.eval()
#     preds = []
#     with torch.no_grad():
#         for text, state_idx, feats in dataloader:  # <-- removed the trailing comma and _
#             text, state_idx, feats = text.to(device), state_idx.to(device), feats.to(device)
#             outputs = model(text, state_idx, feats)
#             predicted = torch.argmax(outputs, dim=1)
#             preds.extend(predicted.cpu().numpy())

#     df_out = pd.DataFrame({'label': preds})
#     df_out.to_csv(output_path, index=False)

In [ ]:
# in your implementation, create the output file using the same format
dic = {"Id": [], "Predicted": []}
for i, pred in enumerate(preds):
    dic["Id"].append(i)
    dic["Predicted"].append(pred)

dic_df = pd.DataFrame.from_dict(dic)
dic_df.to_csv(data_path + "predicted.csv", index=False)

In [ ]:
#baseline
train_vec = []
for doc in tagged_train_data:
    vec = 0
    for w in doc:
        vec += embedding_weights[vocabulary[w]]
    vec = vec / len(doc)
    train_vec.append(vec)

test_vec = []
for doc in tagged_test_data:
    vec = 0
    length = 0
    for w in doc:
        try:
            vec += embedding_weights[vocabulary[w]]
            length += 1
        except:
            continue
    vec = vec / length
    test_vec.append(vec)

clf = LogisticRegression(max_iter=100000000).fit(train_vec, df_train["label"])
preds = clf.predict(test_vec)

Training Word2Vec model...
Model: skip-gram


KeyboardInterrupt: 

In [ ]:
# in your implementation, create the output file using the same format
dic = {"Id": [], "Predicted": []}
for i, pred in enumerate(preds):
    dic["Id"].append(i)
    dic["Predicted"].append(pred)

dic_df = pd.DataFrame.from_dict(dic)
dic_df.to_csv(data_path + "predicted.csv", index=False)